In [1]:
import os
import re
import numpy as np
import pandas as pd
from geopy import Nominatim
from currency_converter import CurrencyConverter
from config import preprocessing as preprocessing_cfg

### Data loading

In [2]:
data_path = './data'
filename = 'otomoto_all_offers_pl_23-04-2023_13-14-37.csv'

In [3]:
df_raw = pd.read_csv(os.path.join(data_path, filename), sep=';', parse_dates=['data_utworzenia_oferty'])
df_raw.head()

C:\Users\szyko\AppData\Local\Temp\ipykernel_3304\3515201177.py:1: DtypeWarning: Columns (192,217,246) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(os.path.join(data_path, filename), sep=';', parse_dates=['data_utworzenia_oferty'])


,id,url,data_utworzenia_oferty,tytul_oferty,cena,waluta,nazwa_sprzedawcy,typ_sprzedawcy,rok_rejestracji_sprzedawcy,lokalizacja,...,drugi_szyberdach_szklany_przesuwny_i_uchylny_el,hardtop,pojemnosc_baterii,autonomia,kierownica_po_prawej_anglik,orurowanie_przednie,fotele_tylne_z_funkcje_masazu,hamulce_z_kompozytow_ceramicznych,opony_off-road,czas_ladowania
0,6106252111,https://www.otomoto.pl/oferta/volvo-v70-podgrz...,2023-04-20 14:52:00,Volvo V70,23200,PLN,Blaupoint Auto Handel,Dealer,2015,"Łużycka 14B - 72-600 Świnoujście, Zachodniopom...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6103725874,https://www.otomoto.pl/oferta/honda-accord-acc...,2023-04-21 06:44:00,Honda Accord 2.0 Comfort,16800,PLN,NaN,Osoba prywatna,2020,"Inowrocław, inowrocławski, Kujawsko-pomorskie",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6109470482,https://www.otomoto.pl/oferta/mercedes-benz-kl...,2023-04-15 14:38:00,Mercedes-Benz Klasa X 350 d 4-Matic Power,249900,PLN,FIRMA HANDLOWO-USŁUGOWA,Dealer,2007,"OLSZYNY-192 - 32-831 Olszyny, gm. Wojnicz, tar...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6108732545,https://www.otomoto.pl/oferta/toyota-avensis-o...,2023-04-13 16:11:00,Toyota Avensis 1.8 VVT-i Sol,16499,PLN,P.H.U ''DaWeKi'',Dealer,2011,"Grodzka 113 - 87-800 Włocławek, Kujawsko-pomor...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6108048263,https://www.otomoto.pl/oferta/ford-c-max-2012r...,2023-04-17 15:15:00,Ford C-MAX 2.0 TDCi Champions Edition,29900,PLN,Auto Komis Kozik,Dealer,2013,"Orchów 172A - 98-100 Łask, łaski, Łódzkie (Pol...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_raw.info(max_cols=len(df_raw.columns))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208205 entries, 0 to 208204
Data columns (total 247 columns):
 #    Column                                               Non-Null Count   Dtype         
---   ------                                               --------------   -----         
 0    id                                                   208205 non-null  int64         
 1    url                                                  208205 non-null  object        
 2    data_utworzenia_oferty                               208205 non-null  datetime64[ns]
 3    tytul_oferty                                         208205 non-null  object        
 4    cena                                                 208205 non-null  int64         
 5    waluta                                               208205 non-null  object        
 6    nazwa_sprzedawcy                                     202912 non-null  object        
 7    typ_sprzedawcy                                       208205 non

### Data cleaning

Check for duplicated rows

In [5]:
duplicates = df_raw.duplicated(subset=df_raw.columns[1:]).sum()
df = df_raw.drop_duplicates(subset=df_raw.columns[1:])
print(f'Removed {duplicates} duplicated offers.')

Removed 0 duplicated offers.


Check the number of missing values in each column

In [6]:
for col in df.columns:
    n_missing = df[col].isnull().sum()
    missing_percent = n_missing / len(df) * 100
    print(f'{col:_<50}: {n_missing:>8} missing values ({missing_percent:.2f}%)')

id________________________________________________:        0 missing values (0.00%)
url_______________________________________________:        0 missing values (0.00%)
data_utworzenia_oferty____________________________:        0 missing values (0.00%)
tytul_oferty______________________________________:        0 missing values (0.00%)
cena______________________________________________:        0 missing values (0.00%)
waluta____________________________________________:        0 missing values (0.00%)
nazwa_sprzedawcy__________________________________:     5293 missing values (2.54%)
typ_sprzedawcy____________________________________:        0 missing values (0.00%)
rok_rejestracji_sprzedawcy________________________:        0 missing values (0.00%)
lokalizacja_______________________________________:        0 missing values (0.00%)
oferta_od_________________________________________:        0 missing values (0.00%)
kategoria_________________________________________:        0 missing values 

Drop columns with irrelevant information and columns almost full of missing values

In [7]:
cols_to_drop = [
    'id',
    'url',
    'tytul_oferty',
    'nazwa_sprzedawcy',
    'rok_rejestracji_sprzedawcy',
    'opis_oferty',
    'oferta_od',
    'kategoria',
    'pokaz_oferty_z_numerem_vin',
    'vat_marza',
    'mozliwosc_finansowania',
    'numer_rejestracyjny_pojazdu',
    'vin',
    'oplata_poczatkowa',
    'miesieczna_rata',
    'liczba_pozostalych_rat',
    'wartosc_wykupu',
    'lub_do_przebieg_km',
    'wersja',
    'srednie_zuzycie',
    'spalanie_w_cyklu_mieszanym',
    'spalanie_poza_miastem',
    'emisja_co2',
    'data_pierwszej_rejestracji_w_historii_pojazdu',
    'pojemnosc_baterii',
    'rodzaj_wlasnosci_baterii',
    'czas_ladowania',
    'autonomia',
]

df = df.drop(columns=cols_to_drop)

Remove damaged cars from the dataset

In [8]:
df['uszkodzony'].value_counts(dropna=False)

NaN    196140
Tak     12065
Name: uszkodzony, dtype: int64

In [9]:
df = df.loc[df['uszkodzony'].isna()]
df = df.drop(columns=['uszkodzony'])

Convert all prices to PLN

In [10]:
df['waluta'].value_counts()

PLN    195811
EUR       329
Name: waluta, dtype: int64

In [11]:
eur_mask = (df['waluta'] == 'EUR')

currency_converter = CurrencyConverter(fallback_on_missing_rate=True, fallback_on_wrong_date=True)

df.loc[eur_mask, 'cena'] = df.loc[eur_mask].apply(lambda x: currency_converter.convert(
    x['cena'], 'EUR', 'PLN', date=x['data_utworzenia_oferty'].date()), axis=1)
df = df.drop(columns='waluta')

Convert car condition column to binary

In [12]:
df['stan'].value_counts()

Używane    175450
Nowe        20690
Name: stan, dtype: int64

In [13]:
df['nowy_pojazd'] = df['stan'].replace({'Nowe': 1, 'Używane': 0}).astype(int)
df = df.drop(columns=['stan'])

Clean and convert numerical columns to numbers

In [14]:
print(df['przebieg'].apply(lambda x: str(x).split()[-1]).value_counts(), end='\n\n')
print(df['pojemnosc_skokowa'].apply(lambda x: str(x).split()[-1]).value_counts(), end='\n\n')
print(df['moc'].apply(lambda x: str(x).split()[-1]).value_counts(), end='\n\n')
print(df['spalanie_w_miescie'].apply(lambda x: str(x).split()[-1]).value_counts(), end='\n\n')
print(df['gwarancja_dealerska_w_cenie'].apply(lambda x: str(x).split()[-1]).value_counts())

km     194808
nan      1332
Name: przebieg, dtype: int64

cm3    192881
nan      3259
Name: pojemnosc_skokowa, dtype: int64

KM     196008
nan       132
Name: moc, dtype: int64

l/100km    113998
nan         82142
Name: spalanie_w_miescie, dtype: int64

nan         186754
miesięcy      9386
Name: gwarancja_dealerska_w_cenie, dtype: int64


In [16]:
df['przebieg'] = df['przebieg'].apply(
    lambda x: float(re.sub(r'\D', '', str(x))) if isinstance(x, str) else None)
df['moc'] = df['moc'].apply(
    lambda x: float(re.sub(r'\D', '', str(x))) if isinstance(x, str) else None)
df['pojemnosc_skokowa'] = df['pojemnosc_skokowa'].apply(
    lambda x: round(float(x.split('cm3')[0].replace(' ', '')) / 1000, 1) if isinstance(x, str) else None)
df['spalanie_w_miescie'] = df['spalanie_w_miescie'].apply(
    lambda x: float(x.split()[0].replace(',', '.')) if isinstance(x, str) else None)

df['gwarancja_dealerska'] = df['gwarancja_dealerska_w_cenie'].apply(
    lambda x: float(x.split()[0]) if isinstance(x, str) else None).fillna(0)
df = df.drop(columns=['gwarancja_dealerska_w_cenie'])

Convert columns with additional car equipment to binary

Fill missing values with 0

Add an indicator for offers without information about additional car equipment

In [17]:
df['brak_informacji_o_wyposazeniu'] = df[preprocessing_cfg.CAR_EQUIPMENT_COLS].isna().all(axis=1).astype(int)
df.loc[:, preprocessing_cfg.CAR_EQUIPMENT_COLS] = df[preprocessing_cfg.CAR_EQUIPMENT_COLS].fillna(value=False).astype(int)

C:\Users\szyko\AppData\Local\Temp\ipykernel_3304\857890836.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, preprocessing_cfg.CAR_EQUIPMENT_COLS] = df[preprocessing_cfg.CAR_EQUIPMENT_COLS].fillna(value=False).astype(int)


Convert remaining True/False columns to binary

Fill missing values with 0

In [18]:
df.loc[:, preprocessing_cfg.BINARY_COLS] = df[preprocessing_cfg.BINARY_COLS].replace('Tak', True).fillna(value=False).astype(int)

C:\Users\szyko\AppData\Local\Temp\ipykernel_3304\4292693481.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, preprocessing_cfg.BINARY_COLS] = df[preprocessing_cfg.BINARY_COLS].replace('Tak', True).fillna(value=False).astype(int)


### Feature extraction

Calculate the age of the cars

In [19]:
df['wiek_pojazdu'] = df['data_utworzenia_oferty'].dt.year - df['rok_produkcji']
df = df.drop(columns=['rok_produkcji'])

Calculate the remaining months of manufacturer's warranty

In [20]:
df['gwarancja_producenta'] = (pd.to_datetime(df['okres_gwarancji_producenta'], format='%d/%m/%Y') -
                              df['data_utworzenia_oferty']) / np.timedelta64(1,'M')
df.loc[df['gwarancja_producenta'] < 0, 'gwarancja_producenta'] = None
df['gwarancja_producenta'] = df['gwarancja_producenta'].round(decimals=1).fillna(0)
df = df.drop(columns=['okres_gwarancji_producenta', 'data_utworzenia_oferty'])

Get province names from offer location addresses

In [21]:
province_names = ['dolnośląskie', 'kujawsko-pomorskie', 'lubelskie', 'lubuskie', 'łódzkie', 'małopolskie',
                  'mazowieckie', 'opolskie', 'podkarpackie', 'podlaskie', 'pomorskie', 'śląskie',
                  'świętokrzyskie', 'warmińsko-mazurskie', 'wielkopolskie', 'zachodniopomorskie']

def get_province_names(locations):
    def find_province_names_in_string(x):
        names = [re.sub(r'[^\w\-]+', '', word.lower(), flags=re.UNICODE) for word in x.split()
                 if re.sub(r'[^\w\-]+', '', word.lower(), flags=re.UNICODE) in province_names]
        return names[0] if names else 'nieznane'

    def find_province_name_in_address(address):
        names = [re.sub(r'[^\w\-]+', '', word.strip(','), flags=re.UNICODE) for word in address.split()
                 if re.sub(r'[^\w\-]+', '', word.strip(','), flags=re.UNICODE) in province_names]
        return names[0] if names else 'nieznane'

    location_provinces = locations.apply(find_province_names_in_string)
    locations_wo_province = locations.loc[location_provinces == 'nieznane']
    unique_locations_wo_province = locations_wo_province.unique()

    missing_provinces = ['nieznane' for _ in unique_locations_wo_province]
    geolocator = Nominatim(user_agent='car_offers_geolocator')

    for i, location_str in enumerate(unique_locations_wo_province):
        location = geolocator.geocode(location_str, addressdetails=True)

        if location is not None:
            location_address = location.address
            province_name = find_province_name_in_address(location_address)
            missing_provinces[i] = province_name

    missing_location_provinces = pd.merge(
        pd.DataFrame({'location': locations_wo_province}),
        pd.DataFrame({'location': unique_locations_wo_province, 'province': missing_provinces}),
        on='location',
        how='inner'
    )['province']
    location_provinces.loc[location_provinces == 'nieznane'] = missing_location_provinces.values

    return location_provinces


df['wojewodztwo'] = get_province_names(df['lokalizacja'])
df = df.drop(columns=['lokalizacja'])

df['wojewodztwo'].value_counts()

mazowieckie            35707
śląskie                22992
wielkopolskie          22348
małopolskie            15618
nieznane               15013
dolnośląskie           12147
łódzkie                11428
pomorskie              11050
kujawsko-pomorskie      8346
lubelskie               7279
podkarpackie            6371
zachodniopomorskie      5811
świętokrzyskie          5598
lubuskie                4957
warmińsko-mazurskie     4535
podlaskie               3635
opolskie                3305
Name: wojewodztwo, dtype: int64

Extract numerical features from the model generation column to avoid encoding high cardinality categorical feature

In [22]:
brand_model_grouped = df.groupby(['marka_pojazdu', 'model_pojazdu'], as_index=False)
models_generations_info = dict()

for (brand, model), subdf in brand_model_grouped:
    model_info = dict()
    n_generations = subdf['generacja'].dropna().nunique()

    if n_generations > 1:
        model_info['liczba_generacji_modelu'] = n_generations
        try:
            model_info['kolejnosc_generacji'] =  sorted(
                list(subdf['generacja'].dropna().unique()),
                key=lambda x: re.findall(r'\((.*?)\)', x)[0].split('-')[0],
                reverse=True
            )
        except IndexError:
            model_info['kolejnosc_generacji'] = []

    else:
        model_info['liczba_generacji_modelu'] = 1
        model_info['kolejnosc_generacji'] = []

    models_generations_info[(brand, model)] = model_info

In [23]:
def get_numbers_of_generations(row, info):
    return info[(row['marka_pojazdu'], row['model_pojazdu'])]['liczba_generacji_modelu']

def get_generation_order(row, info):
    model_generations_order = info[(row['marka_pojazdu'], row['model_pojazdu'])]['kolejnosc_generacji']

    if len(model_generations_order) == 0:
        return 0

    else:
        try:
            return model_generations_order.index(row['generacja'])
        except ValueError:
            return 0


df['liczba_generacji_modelu'] = df.apply(get_numbers_of_generations, axis=1, args=(models_generations_info,))
df['ktora_generacja_modelu'] = df.apply(get_generation_order, axis=1, args=(models_generations_info,))
df = df.drop(columns=['generacja'])

In [25]:
df.to_csv(os.path.join(data_path, 'data_cleaned.csv'), index=False, sep=';')